# Convolutional NN

In [ ]:
def my_conv2d(x, kernel, stride=(1,1), padding=(0,0), bias=None):
    x = x.unsqueeze(0)
    (B, C, H, W) = x.shape
    (C, H_k, W_k) = kernel.shape
    Pad = torch.nn.ZeroPad2d(padding)
    x = Pad(x)
    Hout, Wout = (H + 2*padding[0] - H_k)//stride[0]+1, (W + 2*padding[1] - W_k)//stride[1]+1
    Z = torch.zeros(B, Hout, Wout)
    for h in range(Hout):
        vert_st = h*stride[0]
        vert_end = vert_st + H_k
        for w in range(Wout):
            hor_st = w*stride[1]
            hor_end = hor_st + W_k
            s = x[:, :,vert_st:vert_end, hor_st:hor_end]*kernel
            if bias:
                s += bias
            Z[:,h,w] = torch.sum(s[:])
    return Z.squeeze(0)

In [ ]:
import torch
import torchvision as tv

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
cat_convolve = Image.open('./images/cat.jpg')
plt.imshow(cat_convolve)

In [ ]:
cat_tensor = tv.transforms.ToTensor()(cat_convolve)
print (cat_tensor.size())

### Edges detector
$$
\begin{bmatrix}
    -1       & -1 & -1  \\
    -1       & 8 & -1 \\
    -1       & -1 & -1
\end{bmatrix}
$$

In [ ]:
filter_kernel = torch.FloatTensor(3,3,3).fill_(-1)
filter_kernel[:,1,1] = 8
print(filter_kernel.size())

In [ ]:
o_image = my_conv2d(cat_tensor, filter_kernel)
o_image[o_image >= 0.5] = 1
o_image[o_image < 0.5] = 0
print(o_image.size())
plt.imshow(o_image.numpy(), cmap='gray')

### - horizontal edges
$$
\begin{bmatrix}
    1       & 1 & 1  \\
    0       & 0 & 0 \\
    -1       & -1 & -1
\end{bmatrix}
$$

In [ ]:
filter_kernel = torch.FloatTensor(3,3,3).fill_(1)
filter_kernel[:,1,:] = 0
filter_kernel[:,2,:] = -1
print(filter_kernel.size())

In [ ]:
o_image = my_conv2d(cat_tensor, filter_kernel)
o_image[o_image >= 0.5] = 1
o_image[o_image < 0.5] = 0
print(o_image.size())
plt.imshow(o_image.numpy(), cmap='gray')

### - vertical edges
$$
\begin{bmatrix}
    1       & 0 & -1  \\
    1       & 0 & -1 \\
    1       & 0 & -1
\end{bmatrix}
$$

In [ ]:
filter_kernel = torch.FloatTensor(3,3,3).fill_(1)
filter_kernel[:,:,1] = 0
filter_kernel[:,:,2] = -1
print(filter_kernel.size())

In [ ]:
o_image = my_conv2d(cat_tensor, filter_kernel)
o_image[o_image >= 0.5] = 1
o_image[o_image < 0.5] = 0
print(o_image.size())
plt.imshow(o_image.numpy(), cmap='gray')

# MNIST with CNN

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

In [ ]:
#dataset load
mnist = fetch_openml('mnist_784', data_home='./')

In [ ]:
X = mnist['data'].reshape(-1, 28, 28)
X = X.astype('float')
Y = mnist['target']
Y = np.array(list(map(int, Y)))

In [ ]:
# data normalization
Xm = np.mean(X, axis=0)
Xs = np.std(X, axis=0)
X=(X - Xm) / (Xs + 0.01)

In [ ]:
#train and test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
# data convertion to torch.tensor
X_train = torch.from_numpy(X_train).to(dtype=torch.float32).to(device)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32).to(device)
Y_train = torch.from_numpy(Y_train).to(dtype=torch.long).to(device)
Y_test = torch.from_numpy(Y_test).to(dtype=torch.long).to(device)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1,8,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(8,16,kernel_size=2,stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)            
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size,64),
            nn.ReLU(),
            nn.Linear(64,10))
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.shape))

    def forward(self, x):
        x = self.conv(x).view(x.shape[0],-1)

        return self.fc(x)
    
# minibatches
def iterate_minibatches(X, y, batchsize):
    indices = np.random.permutation(np.arange(len(X)))
    for start in range(0, len(indices), batchsize):
        ix = indices[start: start + batchsize]
        yield X[ix], y[ix]

In [ ]:
# network creation 
net = Net(X_train.unsqueeze(1).shape[1:]).to(device)
# optimizer and loss function selection
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_entropy = nn.CrossEntropyLoss()

In [ ]:
# our NN
print(net)

In [ ]:
num_iteration = 15
batch_size = 256

In [ ]:
L_train = []
L_test = []
test_accuracy = []

for iter in range(num_iteration):

    L = 0.
    # train
    net.train(True)
    for X_batch, y_batch in iterate_minibatches(X_train, Y_train, batch_size):
        loss = loss_entropy(net(X_batch.unsqueeze(1)), y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        L += loss.cpu().detach().numpy()
    L_train.append(L/Y_train.shape[0])
    
    # test
    L = 0.
    net.train(False)    
    for X_batch, y_batch in iterate_minibatches(X_test, Y_test, batch_size):
        with torch.no_grad():
            loss = loss_entropy(net(X_batch.unsqueeze(1)), y_batch)
            L += loss.cpu().numpy()
            
    Y_pred = net.forward(X_test.unsqueeze(1)).cpu().detach().numpy().argmax(1)
    L_test.append(L/Y_test.shape[0])
    
    test_accuracy.append(accuracy_score(Y_pred, Y_test.cpu().numpy()))
    print("{} iter loss. Train : {} . Test : {} Test accuracy {}"
          .format(iter, np.round(L_train[-1],4), np.round(L_test[-1],4), np.round(test_accuracy[-1],3)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (10,5))

ax1.plot(L_train, label='train')
ax1.plot(L_test, label='test')
ax1.grid()
ax1.set_title('Loss')
ax1.legend()

ax2.plot(test_accuracy, label='test')
ax2.grid()
ax2.set_title('Accuracy')
ax2.legend()

plt.show()